In [2]:
import pandas as pd

# Load the CSV files
adp_df = pd.read_csv("2025ADP.csv")
ppr_df = pd.read_csv("2024PPR.csv")

In [3]:
# Merge the DataFrames on the 'Player' column
merged_df = pd.merge(adp_df, ppr_df, on="Player", how="inner")

# Add an 'Available' column
merged_df['Available'] = True

# Extract base position (e.g., 'RB' from 'RB1')
merged_df['BasePOS'] = merged_df['POS'].str.extract(r'([A-Z]+)')
merged_df['BasePOS'] = merged_df['BasePOS'].fillna('UNK')  # assign unknown positions

print(merged_df.head()) 

   Rank            Player Team_x   Bye  POS  Sleeper  RTSports  AVG_x  \
0   1.0     Ja'Marr Chase    CIN  10.0  WR1      1.0       1.0    1.0   
1   2.0    Bijan Robinson    ATL   5.0  RB1      3.0       2.0    2.5   
2   3.0    Saquon Barkley    PHI   9.0  RB2      2.0       4.0    3.0   
3   4.0      Jahmyr Gibbs    DET   8.0  RB3      5.0       3.0    4.0   
4   5.0  Justin Jefferson    MIN   6.0  WR2      4.0       6.0    5.0   

  Real-Time (?)     #  ...    13    14    15    16    17    18 AVG_y    TTL  \
0             1   2.0  ...  20.6  45.1  18.4  22.4  19.2  25.6  23.7  403.0   
1             2   9.0  ...  25.5  20.1  14.5  24.3  24.8  31.3  20.1  341.7   
2             3   8.0  ...  19.7  14.4   9.4    27  18.9     -  22.2  355.3   
3             4   7.0  ...  10.4  19.3  28.4  25.4  26.3    46  21.3  362.9   
4             5  15.0  ...  16.9  32.2  20.3  36.4  17.2   8.4  18.7  317.5   

  Available BasePOS  
0      True      WR  
1      True      RB  
2      True      RB 

In [4]:
def calculate_ValueScore(df):
    df['ValueScore'] = df['TTL'] / df['AVG_x']
    return df

In [5]:
def top_draftable(df, top_n_dict):
    filtered = pd.DataFrame()
    for pos, n in top_n_dict.items():
        top_pos = df[df['BasePOS'] == pos].nlargest(n, 'ValueScore')
        filtered = pd.concat([filtered, top_pos])
    return filtered

In [6]:
def apply_scarcity(df, top_tier_df):
    # Count remaining top-tier players per position
    remaining_counts = top_tier_df[top_tier_df['Available']]['BasePOS'].value_counts().to_dict()
    
    # Count total top-tier per position (for multiplier)
    total_counts = top_tier_df['BasePOS'].value_counts().to_dict()
    
    # Safely calculate adjusted ValueScore
    df['AdjValueScore'] = df.apply(
        lambda row: row['ValueScore'] * (total_counts.get(row['BasePOS'], 1) / remaining_counts.get(row['BasePOS'], 1)),
        axis=1
    )
    
    return df, remaining_counts


In [7]:
# Function to rank players by adjusted value
def rank_players(df):
    return df.sort_values(by='AdjValueScore', ascending=False)


In [8]:
# Function to get top N players
def top_n_players(df, n=10):
    return df.head(n)

In [11]:
def player_unavailable(df, player_name):
    df.loc[df['Player'] == player_name, 'Available'] = False
    return df

In [ ]:
# Initialize ValueScore
merged_df = calculate_ValueScore(merged_df)

# Define top N players per position for scarcity calculation
top_n_per_position = {'QB': 12, 'RB': 25, 'WR': 25, 'TE': 12, 'DST': 10, 'K': 10}
top_tier_df = top_draftable(merged_df, top_n_per_position)

print("Fantasy Draft Picker Running!\n")

# Interactive draft loop
while True:
    # Update top-tier remaining players
    top_tier_df = top_tier_df[top_tier_df['Available']]
    
    # Update scarcity and adjusted ValueScore
    merged_df, remaining_counts = apply_scarcity(merged_df, top_tier_df)
    
    # Get available players and rank them
    available_df = merged_df[merged_df['Available']]
    ranked_df = rank_players(available_df)
    
    # Show top 10 available players
    print("\nTop available players:")
    print(top_n_players(ranked_df, n=10)[['Player', 'POS', 'AdjValueScore']])
    
    # Show remaining counts per position
    print("\nRemaining players per position (top-tier):")
    print(remaining_counts)
    
    # Ask for your draft pick
    player_choice = input("\nEnter the player you want to draft (or type 'done' to finish): ")
    if player_choice.lower() == 'done':
        print("\nDraft finished!")
        break
    
    # Mark the player as unavailable
    merged_df = player_unavailable(merged_df, player_choice)


Fantasy Draft Picker Running!


Top available players:
               Player  POS  AdjValueScore
0       Ja'Marr Chase  WR1     403.000000
1      Bijan Robinson  RB1     136.680000
2      Saquon Barkley  RB2     118.433333
3        Jahmyr Gibbs  RB3      90.725000
4    Justin Jefferson  WR2      63.500000
5         CeeDee Lamb  WR3      47.890909
7   Amon-Ra St. Brown  WR5      35.133333
6        Malik Nabers  WR4      32.188235
10      Derrick Henry  RB6      29.252174
12      De'Von Achane  RB7      22.214815

Remaining players per position (top-tier):
{'RB': 25, 'WR': 25, 'QB': 12, 'TE': 12, 'DST': 10, 'K': 10}

Top available players:
               Player  POS  AdjValueScore
0       Ja'Marr Chase  WR1     403.000000
1      Bijan Robinson  RB1     136.680000
2      Saquon Barkley  RB2     118.433333
3        Jahmyr Gibbs  RB3      90.725000
4    Justin Jefferson  WR2      63.500000
5         CeeDee Lamb  WR3      47.890909
7   Amon-Ra St. Brown  WR5      35.133333
6        Malik Nab